8 应力应变状态分析


拉格朗日力学


In [4]:
import Pkg; Pkg.add("QuadGK")
# Using numerical integration (e.g., QuadGK package)
using QuadGK

# Define a SmoothFunction struct to hold a function and its gradient
struct SmoothFunction
    func::Function
    grad::Function  # Gradient (derivative)
end

# Define a Particle struct with mass, position, velocity, and potential energy
struct Particle
    m::Float64           # Mass
    x::SmoothFunction    # Position as a function of time
    v::SmoothFunction    # Velocity as a function of time
    V::SmoothFunction    # Potential energy as a function of time
end

# Constant value 1/2
const half = 0.5

# Velocity raised to the power n
function v_pow(z::Particle, n::Int)
    new_func = t -> (z.v.func(t))^n
    new_grad = t -> n * (z.v.func(t))^(n-1) * z.v.grad(t)
    SmoothFunction(new_func, new_grad)
end

# Kinetic energy (1/2 * m * v^2)
function Ek(z::Particle)
    m_const = SmoothFunction(t -> z.m, t -> 0.0)  # Constant function for mass
    v_sq = v_pow(z, 2)
    half_const = SmoothFunction(t -> half, t -> 0.0)  # Constant function for 1/2
    # Multiply m_const, v_sq, and half_const
    new_func = t -> m_const.func(t) * v_sq.func(t) * half_const.func(t)
    new_grad = t -> m_const.grad(t) * v_sq.func(t) * half_const.func(t) +
                   m_const.func(t) * v_sq.grad(t) * half_const.func(t) +
                   m_const.func(t) * v_sq.func(t) * half_const.grad(t)
    SmoothFunction(new_func, new_grad)
end

# Lagrangian (Ek - V)
function L(z::Particle)
    ek = Ek(z)
    new_func = t -> ek.func(t) - z.V.func(t)
    new_grad = t -> ek.grad(t) - z.V.grad(t)
    SmoothFunction(new_func, new_grad)
end

# Action (integral of the Lagrangian over time)
function Action(z::Particle, t_start::Float64, t_end::Float64)
    integrand = L(z)
    integral, _ = quadgk(integrand.func, t_start, t_end)
    integral
end

# Derivative of the Lagrangian with respect to velocity (m * v)
function Deriv_L_v(z::Particle)
    new_func = t -> z.m * z.v.func(t)
    new_grad = t -> z.m * z.v.grad(t)
    SmoothFunction(new_func, new_grad)
end

# Derivative of the Lagrangian with respect to position (m * grad V)
function Deriv_L_x(z::Particle)
    new_func = x -> z.m * z.V.grad(x)
    # Assuming grad V is a function of position x
    SmoothFunction(new_func, x -> 0.0)  # Gradient is zero as it's already the derivative
end

# Lagrangian System enforcing the Euler-Lagrange equation
struct LagrangianSystem
    particle::Particle
    # Constructor checks Euler-Lagrange equation
    function LagrangianSystem(particle::Particle)
        # Euler-Lagrange equation: d/dt (dL/dv) = dL/dx
        # Check equality of the gradients
        # Deriv_L_v_grad = derivative of dL/dv with respect to time
        # Deriv_L_x = dL/dx
        # Assuming time derivative of dL/dv is the gradient of dL/dv
        # Compare gradients at sample points (simplified check)
        # For a rigorous check, symbolic differentiation would be needed
        sample_t = 0.0:0.1:10.0  # Sample time points
        for t in sample_t
            el_lhs = Deriv_L_v(particle).grad(t)  # d/dt (dL/dv)
            el_rhs = Deriv_L_x(particle).func(t)   # dL/dx
            if abs(el_lhs - el_rhs) > 1e-6
                error("Euler-Lagrange equation not satisfied at t = $t")
            end
        end
        new(particle)
    end
end

   Resolving package versions...
  No Changes to `C:\Users\danim\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\danim\.julia\environments\v1.11\Manifest.toml`
